In [15]:
import pandas as pd
import h2o
from h2o.estimators import H2OGeneralizedLinearEstimator
from sklearn.datasets import fetch_california_housing

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,25 mins 23 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_nikhilgudur_6nc4cz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.942 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [16]:
cali_housing_data = fetch_california_housing()
cali_housing = pd.DataFrame(cali_housing_data.data, columns=cali_housing_data.feature_names)
cali_housing['median_housing_value'] = cali_housing_data.target

cali_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,median_housing_value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [17]:
cali_housing_h2o = h2o.H2OFrame(cali_housing)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [18]:
train, test = cali_housing_h2o.split_frame(ratios=[.8])

predictors = cali_housing_h2o.columns[:-1]
response = 'median_housing_value'

In [19]:
lmodel = H2OGeneralizedLinearEstimator(family='gaussian')
lmodel.train(x=predictors, y=response, training_frame=train)
predictions = lmodel.predict(test)

predictions


glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
2.42127
2.24395
2.43058
2.14336
2.1529
1.53639
1.66061
1.79078
1.54251
1.39412


In [20]:
MSE = lmodel.model_performance(test_data=test)
MSE

ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.5125769112172494
RMSE: 0.7159447682728391
MAE: 0.5270147636091687
RMSLE: 0.22492077103371
Mean Residual Deviance: 0.5125769112172494
R^2: 0.6071313204344503
Null degrees of freedom: 4064
Residual degrees of freedom: 4056
Null deviance: 5304.564613247287
Residual deviance: 2083.6251440981187
AIC: 8839.312447652523